In [ ]:
import yaml, json
import torch
import os.path as osp

from nphm_tum import env_paths
from nphm_tum.models.neural3dmm import construct_n3dmm, load_checkpoint

In [ ]:
weight_dir_shape = env_paths.EXPERIMENT_DIR_REMOTE + '/'
fname_shape = weight_dir_shape + 'configs.yaml'
with open(fname_shape, 'r') as f:
    print('Loading config file from: ' + fname_shape)
    CFG = yaml.safe_load(f)
    print('###########################################################################')
    print('####################     Model Configs     #############################')
    print('###########################################################################')
    print(json.dumps(CFG, sort_keys=True, indent=4))

    # load participant IDs that were used for training
    fname_subject_index = f"{weight_dir_shape}/subject_train_index.json"
    with open(fname_subject_index, 'r') as f:
        print('Loading subject index: ' + fname_subject_index)
        subject_index = json.load(f)

    # load expression indices that were used for training
    fname_subject_index = f"{weight_dir_shape}/expression_train_index.json"
    with open(fname_subject_index, 'r') as f:
        print('Loading subject index: ' + fname_subject_index)
        expression_index = json.load(f)


    # construct the NPHM models and latent codebook
    device = torch.device("cuda")
    neural_3dmm, latent_codes = construct_n3dmm(
                                  cfg = CFG,
                                  modalities=['geo', 'exp'],
                                  n_latents=[len(subject_index), len(expression_index)],
                                  device=device,
                                  )

    # load checkpoint from trained NPHM model, including the latent codes
    ckpt_path = osp.join(weight_dir_shape, 'checkpoints/checkpoint_epoch_6500.tar')
    print('Loaded checkpoint from: {}'.format(ckpt_path))
    load_checkpoint(ckpt_path, neural_3dmm, latent_codes)

In [ ]:
def forward(self,
                in_dict,
                cond,
                return_grad : bool = False,
                return_can : bool = False,
                skip_color : bool = False,
                ignore_deformations : bool = False,
                ):

In [16]:
geo_mean = latent_codes.codebook['geo'].embedding.weight.mean(dim=0)
geo_std = latent_codes.codebook['geo'].embedding.weight.std(dim=0)
exp_mean = latent_codes.codebook['exp'].embedding.weight.mean(dim=0)
exp_std = latent_codes.codebook['exp'].embedding.weight.std(dim=0)

In [18]:
in_dict = {
    "queries":[
        (0, 0, 0)
    ]
}

cond = {
    "geo": geo_mean,
    "exp": exp_mean
}

In [19]:
neural_3dmm(in_dict, cond)

IndexError: too many indices for tensor of dimension 1